In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!unzip /kaggle/input/sf-crime/train.csv.zip
!unzip /kaggle/input/sf-crime/sampleSubmission.csv.zip
!unzip /kaggle/input/sf-crime/test.csv.zip

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('sampleSubmission.csv')

In [ ]:
display(train,test)

In [ ]:
alldata = pd.concat([train,test])
alldata

In [ ]:
alldata['Dates'] = pd.to_datetime(alldata['Dates'])
alldata['year'] = alldata['Dates'].dt.year
alldata['hour'] = alldata['Dates'].dt.hour
alldata['month'] = alldata['Dates'].dt.month
alldata['day'] = alldata['Dates'].dt.day #월초와 월말의 범죄가 달라집니다. 
# alldata['weekend'] = alldata['D']#주중 주말이 요일이라는 정보가 담고있어서 중복되는 정보가 됩니다. 
alldata['min'] = alldata['Dates'].dt.minute#특정 분에 잘 발생하는 범죄.인터넷 범죄 은행범죄.#인기있는 표 티케팅 #사람마다 관리하는 부서가 다르다. 
# alldata['sec'] = alldata['Dates'].dt.seconds 초가 없어서 중요하지 않습니다. 
#파일이 압축이 언제되었는지, 비슷한 느낌의 데이터는 수집되는 시기가 같기 때문에 index가 유사하기도 합니다. 
alldata['week'] = alldata['Dates'].dt.week #1년을 52주로 나누는게 왜..? 진짜 월초와 월말의 주를 찾을 수 있습니다. 미국데이터는 주단위로 공휴일이 존재하기 때문입니다. 
#범죄도 시간의 흐름에 따라 범죄종류의 발생률이 다르다. 
#시간의 흐름을 추가합시다!!!!!!!!!!!!!!
#특정 범죄가 언제 발생하였는지 숫자적인 느낌으로 학습을 할 수 있습니다. #연도는 범위가 너무 큽니다. 그래서 흐름을 맞춰주겠습니다. 

alldata['time'] = alldata['Dates'].dt.date - alldata['Dates'].dt.date.min() 
alldata#사칙연산이 가능해집니다. 기록이 최초로 되는 시점을 뺍니다. 
alldata['time'] = alldata['time'].apply(lambda x:x.days)#숫자만 쏙 나옵니다. 
alldata

In [ ]:
alldata2 = alldata.drop(columns=['Dates','Category','Id','Resolution','Descript'])#test와 train이 있는 칼럼을 비교할 때 일단은 한쪽에만 있는 칼럼은 제거합니다. 
alldata2

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()


In [ ]:
c = alldata2.columns[alldata2.dtypes==object]

In [ ]:
alldata2.nunique()#각 칼럼의 유니크의 숫자를 셉니다. 

In [ ]:
for i in c:
    alldata2[i] = le.fit_transform(alldata2[i])
alldata2

In [ ]:
alldata2.info()
alldata2.isnull().sum()

In [ ]:
alldata2 = alldata2.fillna(-1)

In [ ]:
train2 = alldata2[:len(train)]
test2 = alldata2[len(train):]

#ranfo의 문제점이 발생합니다. 
- 데이터의 갯수가 많아집니다. 
- 학습해야하는 정보의 양이 많아 데이터가 복잡해졌습니다. 
- 하나의 나무에서 얼마나 학습을 하느냐고 답하냐면 내가 학습을 못 할 때까지 학습하고 넘기겠다. 
- 정보의 이득이 없어질 때까지 학습을 합니다. 
- address는 카테고리의 성향의 데이터인데 종류가 너무 많습니다. 
- 현재 중요한 address의 카테고리가 ranfo()를 사용하려면  max_depth로 조정을 해줘야합니다. 
####중요한칼럼에 숫자가 많고, 데이터가 많으면 램이 터집니다. 데이터가 많고 학습을 많이 해야하기 때문입니다. 

#부스팅모델은 트리형보다 옵션값을 잘 주지 못한다면 부스팅이 좋더라도 트리형이 더 잘 나올 수 있습니다. 

부스팅모델의 종류
- catboosting은 카테고리적인 형태, 옵션값의 설정을 다른 부스팅모델보다 적다. cat = category입니다.


randomforest는 gradient를 찾아 떠나는 활용을 하지 않습니다. 

In [ ]:
from sklearn.model_selection import train_test_split
train_x, valid_x, train_y,valid_y = train_test_split(train2,train['Category'], test_size=0.2,stratify=train['Category'],random_state=42)
#random_state=42를 지정하지 않으면 평가셋이 달라진다. 정말 좋아지는지를 봐야하는데 그때그때 달라진다.
#뽑힌 평가셋이 운이 좋아진 건지 명확히 알고자 하려고. 대조군을 명확히 한다. 
#안정적인 평가셋을 만들어주기 위해서 앞으로 들어올 새로운 데이터를 대비합니다. 
#평가셋이 새로운 데이터의 분포를 그대로 따라할 수 있도록 돕습니다. 
#20%씩 넣어진다.#오른쪽은 정답값 
#train과 testset에서 도움이 안된다면..두둥..... 평가셋을 만들때도 정답 클래스의 비율을 맞춰준다. 

In [ ]:
train_x.shape

In [ ]:
from catboost import CatBoostClassifier
cbf = CatBoostClassifier(task_type='GPU')#gpu로 돌리겠다 #n_jobs를 쓰지 않아도 전제조건이 전체를 돌린다는 의미입니다. 

In [ ]:
cbf.fit(train_x, train_y,eval_set=(valid_x,valid_y)) # 평가셋을 넣어줍니다. 
result = cbf.predict_proba(test2) #epoch 같은 개념입니다. 이터레이션과 같은 의미입니다...ㅎㅎ
#중앙에 값이 생겼습니다. 모델의 진짜 점수입니다. 

In [ ]:
#분류문제는 logloss를 확인합니다. 

In [ ]:
sub.iloc[:,1:] = result 
sub

In [ ]:
sub.to_csv('crime.csv',index=0)#데이터가 커서 시간이 많이 걸리는 문제가 됩니다.